In [ ]:
#CLASIFICADOR

# Instalación de dependencias
!pip install gradio
!pip install transformers
!pip install torch
!pip install sentencepiece

import gradio as gr
from transformers import (
    pipeline,
    AutoModelForSequenceClassification,
    AutoTokenizer
)
import torch
import numpy as np

# Configurar el dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"

# === CONFIGURACIÓN DEL CHATBOT ===
chat_generator = pipeline(
    'text-generation',
    model='microsoft/DialoGPT-small',
    device=device
)

# === CONFIGURACIÓN DEL ANALIZADOR DE SENTIMIENTOS ===
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
sentiment_tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(model_name)
sentiment_model.to(device)

def chatbot(mensaje):
    if not mensaje.strip():
        return "Por favor, escribe un mensaje.", None, None

    try:
        # Generar respuesta del chatbot
        respuesta = chat_generator(
            mensaje,
            max_length=100,
            temperature=0.7,
            do_sample=True,
            top_p=0.95,
            num_return_sequences=1
        )[0]['generated_text']

        respuesta = respuesta.replace(mensaje, "").strip()

        # Analizar sentimiento del mensaje del usuario
        inputs = sentiment_tokenizer(mensaje, return_tensors="pt", truncation=True,
                                   max_length=512, padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        outputs = sentiment_model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        rating = torch.argmax(predictions).item() + 1
        confidence = predictions[0][rating-1].item()

        if rating < 2:
            sentimiento = "Muy Negativo"
        elif rating == 2:
            sentimiento = "Negativo"
        elif rating == 3:
            sentimiento = "Neutral"
        elif rating == 4:
            sentimiento = "Positivo"
        else:
            sentimiento = "Muy Positivo"

        sentimiento_completo = f"{sentimiento} ({rating} estrellas)"
        confianza = round(confidence * 100, 2)

        return respuesta, sentimiento_completo, confianza

    except Exception as e:
        return f"Error: {str(e)}", "Error en el análisis", 0.0

# Crear la interfaz
demo = gr.Interface(
    fn=chatbot,
    inputs=[
        gr.Textbox(
            placeholder="Escribe tu mensaje...",
            label="Mensaje",
            lines=3
        )
    ],
    outputs=[
        gr.Textbox(label="Respuesta del Chatbot"),
        gr.Label(label="Sentimiento de tu mensaje"),
        gr.Number(label="Confianza del análisis (%)")
    ],
    title="Chatbot con Análisis de Sentimientos",
    description="Un chatbot que responde a tus mensajes y analiza el sentimiento de lo que escribes",
    examples=[
        ["¡Estoy muy feliz hoy!"],
        ["No me gusta nada este servicio, es terrible."],
        ["El día está normal, nada especial que contar."],
        ["¡Me encanta hablar contigo!"],
        ["Estoy un poco decepcionado con los resultados."]
    ],
    allow_flagging="never",
    cache_examples=True
)

# Lanzar la interfaz
demo.launch(share=True, debug=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Caching example 4/5


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Caching example 5/5
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://52b9c38593710de8fe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
